<p style="color:#211f78"> 
<center><font size= 6> <b> COMPARACIÓN AUTOMATIZADA DE PRECIOS DE PRODUCTOS DE PRIMERA NECESIDAD EN LOS 3 SUPERMERCADOS MÁS POPULARES: METRO, TOTTUS y PLAZA VEA. </b> </font> </center> 
</p>

<p style="color:#63381f"><font size= 5.5> <b>Integrantes:</b></font> </p> 

<font size= 3.5>
    
<p style="color:#63381f">   
- Tanta Medina, Christian Rodolfo 20200338
</p> 
    
<p style="color:#63381f">
- Taquiri Pillaca, Jorge Ignacio 20200339
</p> 
    
<p style="color:#63381f">
- Tay Saccaco, Brian Eduardo
 </p> 
    
</font>

## DESCRIPCIÓN DEL CASO

Se quiere automatizar la comparación de precios de productos básicos de la canasta familiar
(arroz, azúcar, aceite, leche, etc) de tres supermercados que ofrecen sus productos a través de
su propio portal web. Estos supermercados son: Metro, Plaza Vea y Tottus.
Por lo que se pide escribir un programa en Python utilizando Jupyter notebook que recopile los
precios de un producto de la canasta básica familiar de 3 Supermercados anteriormente
mencionados y reporte todos sus artículos relacionados con la palabra clave ingresada en la
búsqueda, para que posteriormente pueda comparar los precios en los distintos
Supermercados de los productos exactamente iguales. Además, que se envíe
automáticamente al correo del usuario-consultor un reporte de precio más alto, precio más
bajo, productos en común comparando los precios entre los supermercados coincidentes.

# Importar librerías


In [9]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
from termcolor import colored
import csv
from bs4 import BeautifulSoup
import math

pd.set_option('display.max_columns', None)  
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.width', None)
                      
#jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

C:\Users\jorge\AppData\Local\Temp\ipykernel_10544\1633562002.py:19: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


# Ingresar producto y correo 

In [8]:
busqueda=str(input("Coloque el producto a buscar:"))             # Ingreso del producto (por ejemplo: azucar)
para_email= input("Enviar comparación de productos al correo: ") # Email de destino

Coloque el producto a buscar:azucar
Enviar comparación de productos al correo: 20200339@lamolina.edu.pe


<p style="color:#deb03e"> <font size=6.5> <b> Programa para obtener los productos y precios de supermercados Metro (Usuario Jorge) </b> </font> </p>  

In [10]:
urls='https://www.metro.pe'          # URL de la pagina de Metro                   
options = webdriver.EdgeOptions()    # Opciones del navegador 
options.add_argument('--incognito')  # Opcion de abrir ventana de Edge en modo incognito
options.add_argument("--start-maximized")  # ventana maximizada 
s = Service(EdgeChromiumDriverManager().install()) # Esta opcion nos permite usar el ejecutable Edge sin necesidad de colocar su ubicación                     
driver = webdriver.Edge(service=s, options = options)
driver.implicitly_wait(4) # Esperar 4 seg al buscar un elemento que está presente en la pagina       
driver.get(urls) # Abrir el navegador con la url

time.sleep(2)   # Esperar 2 segundos

In [11]:
# Para cancelar las notificaciones de Metro (ventana emergente)
WebDriverWait(driver, 25)\
    .until(EC.element_to_be_clickable((By.CSS_SELECTOR,
                                      '#onesignal-slidedown-cancel-button'.replace(' ', '.'))))\
    .click()

In [12]:
# Hacer clic en "Si acepto", pagina emergente sobre "Políticas general de Cookies"
driver.find_element(By.XPATH, '/html/body/div[46]/div/div[3]/button[1]').click()
time.sleep(1)

In [13]:
# Hacer clic en la barra de búsqueda
driver.find_element(By.XPATH, '/html/body/header[1]/div/div[2]/div[2]/div[1]/div[1]/input').click()
time.sleep(1)

In [14]:
# Ingresar el producto en la barra de busqueda y realizar la busqueda (+ keys.ENTER simula el boton de ENTER)
driver.find_element(By.XPATH, '/html/body/header[1]/div/div[2]/div[2]/div[1]/div[1]/input').send_keys(busqueda + Keys.ENTER)
time.sleep(1)

In [15]:
# Cantidad de productos en la pagina.
page=driver.find_element(By.XPATH, '/html/body/div[27]/div/div[2]/div[1]/div[1]/span[2]').text
print(page)
#int(page[:-10])

162 productos


In [22]:
# Automatización de scrolls (se correra las veces que sea necesario) 
scrolls = 5 + 1          # Seran 5 scrolls automaticos
while True:               # Mientras esto sea verdad se realizará lo siguiente:
    scrolls -= 1          # Cuenta regresiva, se resta 1 cada ciclo
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)") # Se desplazará hasta la parte inferior de la página 
    time.sleep(1)         # Cada scroll automático tendra un timelap de 1 segundo
    if scrolls < 0:       # Cuando llegue a cero segundos:
        time.sleep(2)     # Le damos 2 segundos a que la pagina cargue los contenidos hasta el ultimo scroll automatico realizado
        elements = driver.find_elements(By.XPATH,'//li[@layout="19ccd66b-b568-43cb-a106-b52f9796f5cd"]') # Encontrar los productos de la pagina de acuerdo al XPATH
        break             # Se cierra el ciclo

        
print(colored('Productos extraidos:','red'))  
print(len(elements))      # Longitud de los elementos (productos) encontrados hasta coindicir con el total de productos
print('\n')
print(colored('Se llegó a extraer todos los productos?','yellow'))  
print(len(elements) == int(page[:-10])) # Comparación 

Productos extraidos:
162


Se llegó a extraer todos los productos?
True


In [24]:
# Almacenar productos y precios de Metro 
productos1 =  []   # lista vacia para productos de Metro
precios1 = []      # lista vacia para precio de Metro
supermercado1 = [] 
for element in tqdm(elements):  # tqdm:Contar iteraciones, indicar el tiempo que tarda por iteración y el tiempo total transcurrido.  
        producto = element.find_element(By.XPATH,'.//a[@class="product-item__name"]').text  # Buscar producto en base al XPATH y extraer el texto
        precio = element.find_element(By.XPATH,'.//span[@class="product-prices__value product-prices__value--best-price"]').text # Buscar precio en base al XPATH y extraer el texto
        productos1.append(producto)    # Agregar a la lista productos1
        precios1.append(precio)        # Agregar a la lista precios1
        supermercado1.append('Metro')


100%|██████████| 162/162 [00:21<00:00,  7.37it/s]


In [25]:
#Exportacion de productos y precios a un archivo excel 
dic= {'Supermercado': supermercado1 ,'Producto': productos1 , "Precio": precios1}  # Crear diccionario con los values productos y precios (listas)
df = pd.DataFrame(dic , columns= ['Supermercado', "Producto", "Precio"] ) # Convertir a dataframe con columnas de etiquetas Producto y Precio
df.to_excel('metro.xlsx', index=False,encoding='cp1252')  # Exportar a excel

# Lectura y muestra de resultados
metro = pd.read_excel('metro.xlsx')
metro

,Supermercado,Producto,Precio
0,Metro,Azúcar Rubia Cuisine&Co Bolsa 5 kg,S/26.50
1,Metro,Azúcar Rubia Dulfina Bolsa 2 kg,S/10.90
2,Metro,Azúcar Rubia Costeño 1 kg,S/5.90
3,Metro,Azúcar Impalpable Universal 200 g,S/3.10
4,Metro,Azúcar Blanca Dulfina Bolsa1 kg,S/6.80
5,Metro,Azúcar Rubia Onza Sachet 5 g Pack 500 unid,S/15.45
6,Metro,Azúcar Blanca Con Stevia Cuisine&Co Doypack 500 g,S/7.80
7,Metro,Azúcar Blanca Refinada Dulfina Bolsa 2 kg,S/13.10
8,Metro,Azúcar Rubia Paramonga Bolsa 5 kg,S/28.55
9,Metro,Azúcar Rubia Máxima Bolsa 1 kg,S/5.10


In [26]:
driver.close()  # Cerrar chromedriver

<p style="color:#059a38"> <font size=6.5> <b> Programa para obtener los productos y precios de supermercados Tottus (Usuario Christian) </b> </font> </p>   <font size=6.5> <b> 

In [19]:
urls='https://www.tottus.com.pe/'    # URL de la pagina Tottus
options = webdriver.EdgeOptions()    # Opciones del navegador 
options.add_argument('--incognito')  # Opcion de abrir ventana de Edge en modo incognito
options.add_argument("--start-maximized") # ventana maximizada 
s = Service(EdgeChromiumDriverManager().install()) # Esta opcion nos permite usar el ejecutable Edge sin necesidad de colocar su ubicación                      
driver = webdriver.Edge(service=s, options = options) 
driver.implicitly_wait(4)  # Esperar 4 seg al buscar un elemento que está presente en la pagina
driver.get(urls) # Abrir el navegador con la url

time.sleep(2)   # Esperar 2 segundos

In [20]:
#Al iniciar y entrar a la pagina de Tottus nos salta un anuncio donde Tottus nos quiere informar sobre sus mejores ofertas,
#con este codigo automatizamos dando click en cancelar. 
WebDriverWait(driver, 25)\
    .until(EC.element_to_be_clickable((By.CSS_SELECTOR,
                                      'button.align-right secondary slidedown-button'.replace(' ', '.'))))\
    .click()

In [21]:
# Cerrando ventana de mensaje de anuncios de oferta por agosto (sale de vez en cuando por ello esta comentarizada)
# driver.find_element(By.XPATH, '/html/body/div[3]/div/div/section/div[1]/div/button').click()
# time.sleep(1)

In [22]:
# Configurando click para búsqueda de productos
driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/section/div[1]/div/input').click()
time.sleep(1)

In [23]:
# Ingresando producto + ENTER automatico
driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/section/div[1]/div/input').send_keys(busqueda + Keys.ENTER)
time.sleep(1)

In [24]:
#Obtener el número de paginas
page = driver.find_element(By.XPATH, '/html/body/div[1]/section/div[1]/section/div[2]/div[1]/div[2]/section/div[1]/div[1]/p').text
print(page)
pag = int(page[1:4])/47  # Dividido por la cantidad de productos en un pagina
import math
paginas = math.ceil(pag)  # Redondeo arriba
paginas

(188 resultados)


4

In [25]:
# Extraer todos los productos por cada pagina
productos2 = []
precios2 =[]
supermercado2 = []
n=1
while n <= paginas:
    driver.get(f'https://www.tottus.com.pe/buscar?q={busqueda}&page={n}') # Se ira iterando de acuerdo a la cantidad de paginas empezando por la pagina 1
    time.sleep(2)  # Lapso de tiempo de 2 segundos
    lista = driver.find_elements(By.XPATH,'//li[@class="product"]') # Almacenar los productos encontrados en las paginas
    for element in tqdm(lista):  
        name = element.find_element(By.XPATH,'.//h2[@class="jsx-369422875 name title mini"]').text # Extraer nombre del producto
        price = element.find_element(By.XPATH,'.//span[@class="jsx-4019956066 "]').text # Extraer el precio del producto
        productos2.append(name) 
        precios2.append(price)
        supermercado2.append('Tottus')
    #lista=list.append(lista)
    n += 1


100%|██████████| 44/44 [00:03<00:00, 13.47it/s]


In [26]:
print(colored('Se llegó a extraer todos los productos?','green')) # Mensaje
print(len(productos2) >= int(paginas)) # Comparación 

Se llegó a extraer todos los productos?
True


In [27]:
# Exportación y lectura del archivo excel
dic= {'Supermercado':supermercado2 , 'Producto': productos2,"Precio": precios2 }
df = pd.DataFrame(dic , columns= ['Supermercado',"Producto","Precio"] )
df.to_excel('tottus.xlsx', index=False,encoding='cp1252')

# Lectura
tottus = pd.read_excel('tottus.xlsx')
tottus

,Supermercado,Producto,Precio
0,Tottus,AZÚCAR RUBIA TOTTUS 5 KG,s/ 19.9 UN
1,Tottus,AZÚCAR RUBIA DULFINA 5 KG,s/ 22.9 UN
2,Tottus,AZÚCAR RUBIA TOTTUS 1 KG,s/ 4.7 UN
3,Tottus,AZÚCAR RUBIA TOTTUS 2 KG,s/ 8.5 UN
4,Tottus,AZÚCAR RUBIA DULFINA 1 KG,s/ 5.2 UN
5,Tottus,AZÚCAR BLANCA DULFINA 1 KG,s/ 5.9 UN
6,Tottus,AZÚCAR BLANCA TOTTUS 2 KG,s/ 8.5 UN
7,Tottus,AZÚCAR RUBIA DULFINA 2 KG,s/ 9.5 UN
8,Tottus,BEBIDA DE COCO SIN AZÚCAR TERRAFERTIL 946 ML,s/ 11.9 UN\ns/ 12.5 UN
9,Tottus,AZÚCAR BLANCA CON STEVIA TOTTUS 500 GR,s/ 7.5 UN


In [28]:
driver.close()  # Cerrar chromedriver

<p style="color:#fc0313"> <font size=6.5> <b> Programa para obtener los productos y precios de supermercados Plaza Vea  (Usuario Brian)  </b> </font> </p>   <font size=6.5> <b>

In [29]:
url='https://www.plazavea.com.pe'    # URL de la pagina de Plaza Vea
options = webdriver.EdgeOptions()    # opciones del naegador 
options.add_argument('--incognito')  # Opcion de abrir ventana de Edge en modo incognito
options.add_argument("--start-maximized") # ventana maximizada 
s = Service(EdgeChromiumDriverManager().install())   # Usan Edgedriver sin necesidad de colocar ubicacion                     
driver = webdriver.Edge(service=s, options = options) 
driver.implicitly_wait(4) # Espera de 4 segundos entre una busqueda y otra
driver.get(url)
time.sleep(2)

In [30]:
#Al iniciar y entrar a la pagina salta una ventana emergente acerca de las notificaciones,
#con este codigo automatizamos dando click en cancelar. 
WebDriverWait(driver, 25)\
    .until(EC.element_to_be_clickable((By.CSS_SELECTOR,
                                      '#onesignal-slidedown-cancel-button'.replace(' ', '.'))))\
    .click()

In [31]:
# Click en la barra de busqueda
driver.find_element(By.XPATH, '/html/body/div[3]/header/div[2]/div/div[4]/section/div[2]/div/div/input').click()

In [32]:
# Realizar busqueda y ENTER 
driver.find_element(By.XPATH, '/html/body/div[3]/header/div[2]/div/div[4]/section/div[2]/div[2]/div[1]/input').send_keys(busqueda + Keys.ENTER)

In [33]:
# Encontrando el total de productos y el numero de paginas
page=driver.find_element(By.XPATH, '/html/body/div[4]/main/div/div[1]/div[2]/div[3]/div/div[1]/div/h2/span[2]').text
print(page)               # Total de productos
pag = int(page[1:4])/46
import math
paginas = math.ceil(pag)  # Redondeo arriba
paginas

(52 productos)


2

In [34]:
# Extrayendo los productos y sus precios
productos3 = []
precios3 =[]
supermercado3 = []
n=1

while n <= paginas:
    driver.find_element(By.CSS_SELECTOR, f'#category-page > div.vitrine.resultItemsWrapper > div > div.paginator-container > div > div > span:nth-child({n})').click()
    time.sleep(3)
    lista = driver.find_elements(By.XPATH,'//div[@data-stock="true"]')
    for element in tqdm(lista):
        name = element.find_element(By.XPATH,'.//a[@class="Showcase__name"]').text
        price = element.find_element(By.XPATH,'.//span[@class="price"]').text
        productos3.append(name)
        precios3.append(price)
        supermercado3.append('Plaza Vea')
    n += 1   

100%|██████████| 2/2 [00:00<00:00, 17.73it/s]


In [35]:
print(colored('Se llegó a extraer todos los productos?','red'))  
print(len(productos3) >= int(paginas)) # Comparación 

Se llegó a extraer todos los productos?
True


In [36]:
# Exportar a excel
dic= {'Supermercado':supermercado3 , 'Producto': productos3,"Precio": precios3 }
df = pd.DataFrame(dic , columns= ['Supermercado',"Producto","Precio"] )
df.to_excel('plazavea.xlsx', index=False,encoding='cp1252')
# Lectura
plazavea=pd.read_excel('plazavea.xlsx')
plazavea

,Supermercado,Producto,Precio
0,Plaza Vea,Azúcar Rubia BELL'S Bolsa 4Kg,S/ 16.70 un
1,Plaza Vea,Azúcar Rubia BELL'S Selección Bolsa 5Kg,S/ 23 un
2,Plaza Vea,Azúcar Rubia BELL'S Bolsa 1.3Kg,S/ 5.70 un
3,Plaza Vea,Azúcar Rubia DULFINA Bolsa 5Kg,S/ 21.90 un
4,Plaza Vea,Azúcar Blanca BELL'S Bolsa 4Kg,S/ 17.90 un
5,Plaza Vea,Azúcar Blanca BELL'S Bolsa 1.3Kg,S/ 5.90 un
6,Plaza Vea,Azúcar Rubia BELL'S Bolsa 650g,S/ 3.10 un
7,Plaza Vea,Azúcar Rubia BELL'S Selección Bolsa 1K,S/ 5.30 un
8,Plaza Vea,Azúcar Rubia BELL'S Bolsa 5Kg,S/ 19.90 un
9,Plaza Vea,Azúcar Rubia COSTEÑO Bolsa 1Kg,S/ 5.90 un


In [41]:
driver.close()  # Cerrar chromedriver

# Comparar productos de acuerdo a una búsqueda en común usando expresiones regulares

<b> Supuesto: </b> Azúcar Rubia de 1kg

In [69]:
# Usamos regex de acuerdo al supuesto, y realizar un conteo de los resultados de acuerdo al patron de busqueda
df_metro = metro[metro['Producto'].str.count(r'(Azúcar.*Rubia.*1.*)')==1].sort_values(by='Precio', ascending = True) # Ordenamos los resultados de menor a mayor
df_metro=df_metro.reset_index(drop=True) # Resetear los indices (index)
df_metro # Visualizar dataframe df_metro

,Supermercado,Producto,Precio
0,Metro,Azúcar Rubia Máxima Bolsa 1 kg,S/5.10
1,Metro,Azúcar Rubia Metro Bolsa 1 kg,S/5.30
2,Metro,Azúcar Rubia Cuisine&Co Bolsa 1 kg,S/5.70
3,Metro,Azúcar Rubia Dulfina Bolsa 1 kg,S/5.70
4,Metro,Azúcar Rubia Paramonga 1kg,S/5.85
5,Metro,Azúcar Rubia Costeño 1 kg,S/5.90


In [76]:
df_tottus = tottus[tottus['Producto'].str.count(r'(AZÚCAR.*RUBIA.*1.*)')==1].sort_values(by='Precio', ascending = True)# Ordenamos los resultados de menor a mayor
df_tottus=df_tottus.reset_index(drop=True) # Resetear los indices (index)
df_tottus # Visualizar dataframe df_tottus

,Supermercado,Producto,Precio
0,Tottus,AZÚCAR RUBIA TOTTUS 1 KG,s/ 4.7 UN
1,Tottus,AZÚCAR RUBIA DULFINA 1 KG,s/ 5.2 UN


In [77]:
df_plazavea= plazavea[plazavea['Producto'].str.count(r'(Azúcar.*Rubia.*1.*)')==1].sort_values(by='Precio', ascending = True) # Ordenamos los resultados de menor a mayor
df_plazavea = df_plazavea.reset_index(drop=True)  # Resetear los indices (index)
df_plazavea # Visualizar dataframe df_plazavea

,Supermercado,Producto,Precio
0,Plaza Vea,Azúcar Rubia BELL'S Bolsa 1Kg,S/ 4.49 un
1,Plaza Vea,Azúcar Rubia CARTAVIO Bolsa 1Kg,S/ 4.80 un
2,Plaza Vea,Azúcar Rubia DULFINA Bolsa 1Kg,S/ 4.89 un
3,Plaza Vea,Azúcar Rubia BELL'S Selección Bolsa 1K,S/ 5.30 un
4,Plaza Vea,Azúcar Rubia BELL'S Bolsa 1.3Kg,S/ 5.70 un
5,Plaza Vea,Azúcar Rubia COSTEÑO Bolsa 1Kg,S/ 5.90 un


# Finalmente 


In [64]:
comparacion = pd.concat([df_metro,df_tottus,df_plazavea]) # Concatenamos los 3 dataframes (df_metro,df_tottus,df_plazavea)
comparacion # Visualizar comparación

,Supermercado,Producto,Precio
0,Metro,Azúcar Rubia Máxima Bolsa 1 kg,S/5.10
1,Metro,Azúcar Rubia Metro Bolsa 1 kg,S/5.30
2,Metro,Azúcar Rubia Cuisine&Co Bolsa 1 kg,S/5.70
3,Metro,Azúcar Rubia Dulfina Bolsa 1 kg,S/5.70
4,Metro,Azúcar Rubia Paramonga 1kg,S/5.85
5,Metro,Azúcar Rubia Costeño 1 kg,S/5.90
0,Tottus,AZÚCAR RUBIA TOTTUS 1 KG,s/ 4.7 UN
1,Tottus,AZÚCAR RUBIA DULFINA 1 KG,s/ 5.2 UN
0,Plaza Vea,Azúcar Rubia BELL'S Bolsa 1Kg,S/ 4.49 un
1,Plaza Vea,Azúcar Rubia CARTAVIO Bolsa 1Kg,S/ 4.80 un


In [79]:
# Exportando a excel las comparaciones 3 hojas distintas
writer = pd.ExcelWriter('Reporte.xlsx') # Reporte.xlsx con opcion a escritura

df_metro.to_excel(writer, 'Metro', index=False) # Guardamos el dataframe df_metro en una hoja llamada Metro
df_tottus.to_excel(writer, 'Tottus', index=False) # Guardamos el dataframe df_tottus en una hoja llamada Tottus
df_plazavea.to_excel(writer, 'Plaza Vea', index=False) # Guardamos el dataframe df_plazavea en una hoja llamada Plaza Vea

writer.save() # Guardar los cambios
writer.close() # Cerrar 

# Exportando a csv 
comparacion.to_csv('Reporte.csv', index=False,encoding='cp1252') 

# Envío de correo automático

In [59]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
 
remitente = 'jorgeigna.2911@gmail.com' 
destinatarios = para_email                  # Destinatario (se colocó en un principio)
asunto = '[RESULTADOS] TRABAJO FINAL LP2'
cuerpo = '''
Hola estimado usuario, le enviamos las comparaciones de los productos.
Integrantes:
- Tanta Medina, Christian Rodolfo 20200338
- Taquiri Pillaca, Jorge Ignacio 20200339
- Tay Saccaco, Brian Eduardo 
'''
ruta_adjunto = 'Reporte.xlsx'
nombre_adjunto = 'Comparacion de productos.xlsx'

mensaje = MIMEMultipart() # Creamos el objeto mensaje
# Establecemos los atributos del mensaje
mensaje['From'] = remitente
mensaje['To'] = ", ".join(destinatarios)  # . join en caso haya mas de 1 destinatario
mensaje['Subject'] = asunto

# Agregamos el cuerpo del mensaje como objeto MIME de tipo texto
mensaje.attach(MIMEText(cuerpo, 'plain')) 
# Abrimos el archivo que vamos a adjuntar
archivo_adjunto = open(ruta_adjunto, 'rb') # "rb" abre el archivo en formato binario para lectura
adjunto_MIME = MIMEBase('application', 'octet-stream') # Creamos un objeto MIME base 
adjunto_MIME.set_payload((archivo_adjunto).read()) # Cargamos el archivo adjunto
encoders.encode_base64(adjunto_MIME) # Codificamos el objeto en BASE64
adjunto_MIME.add_header('Content-Disposition', "attachment; filename= %s" % nombre_adjunto) # Agregamos una cabecera al objeto
mensaje.attach(adjunto_MIME) # Finalmente lo agregamos al mensaje

# Creamos la conexión con el servidor
sesion_smtp = smtplib.SMTP('smtp.gmail.com', 587) 
sesion_smtp.starttls() # Ciframos la conexión
sesion_smtp.login('jorgeigna.2911@gmail.com','bnriqogwriiscvoi') # Iniciamos sesión en el servidor
texto = mensaje.as_string() # Convertimos el objeto mensaje a texto
# Enviamos el mensaje
sesion_smtp.sendmail(remitente, destinatarios, texto)
# Cerramos la conexión
sesion_smtp.quit()  

(221,
 b'2.0.0 closing connection g12-20020a1f200c000000b00378fe8518dcsm3105337vkg.51 - gsmtp')

# Envio

![foto](captura1.png "")

[](usuario.png)